<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/instagramm_cookes_mail_22022019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Получение фото

In [0]:
import hashlib
import json
import re
import requests


count=36
session = requests.Session()

      
def update(obj=None, pointer=None, idx=None, rhx_gis=None, settings={}):

      src_adress = [] 
      base_url = ""
      query = "https://www.instagram.com/"
      if not obj is None:
            query += base_url + obj
            
        
      if pointer is None: 
        response = session.get(query, **settings)
        match = re.search(r"<script[^>]*>\s*window._sharedData\s*=\s*((?!<script>).*)\s*;\s*</script>", response.text)
        data = json.loads(match.group(1))
        rhx_gis = data["rhx_gis"]
        csrf_token = data["config"]["csrf_token"]
        idx = data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["id"]
        
        #print("end_cursor", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor'])
        #print("'has_next_page", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page'])
        
        for number in range(len(data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"])):
	          src_adress.append(data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"][number]["node"]["thumbnail_resources"][4]["src"])
        if data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']:                 
            pointer = data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        else:
            pointer = None

        return rhx_gis, pointer, src_adress, idx, csrf_token

      
      else:
        variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
        while True:
            data = {"id": idx, "first": 36, "after": pointer}
            variables=variables_string.format(**data)
            query_hash="c6809c9c025875ac6f02619eae97a80e"
            #response = graphql_request(query_hash="c6809c9c025875ac6f02619eae97a80e", variables=variables, settings=settings)
            settings["params"] = {"query_hash": query_hash}
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            response = session.get("https://www.instagram.com/graphql/query/", **settings)
            response.raise_for_status()
            
            data = response.json()["data"]["user"]["edge_owner_to_timeline_media"]

            edges = data["edges"]
            page_info = data["page_info"]
            length = len(edges)

            for index in range(length):
                    node = edges[index]["node"]
                    shortcode = node["shortcode"]        #["src"]
                    if not "shortcode" in node:
                        length -= 1
                        continue
                    m = node["display_url"]
                    #m._set_data(node)
                    src_adress.append(m)
                    #print("node", node)
                
            if page_info["has_next_page"]:
                    
                    pointer = page_info["end_cursor"]
            else:
                    pointer = None

            return src_adress, pointer

      

In [0]:
src_adress = []
rhx_gis, pointer, src, idx, csrf_token = update("likekate_")
src_adress.extend(src)
print("rhx_gis:", rhx_gis, "csrf_token:", csrf_token, "idx =", idx)
for i in range(100):
  src, pointer = update("likekate_", pointer=pointer, idx=idx, rhx_gis=rhx_gis)
  src_adress.extend(src)
  print(len(src_adress))


# Авторизация

In [0]:
import hashlib
import json
import pickle
import re
import requests
session = requests.Session()
# rhx_gis: fc2e73d4fd7dddcd31d28bea5cb2df59 csrf_token: 23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv
def AgentAccount(login=None, password=None, settings={}, load=False):
    # load cookie    
        if load:
           print("1")
           LOGIN_URL = "https://www.instagram.com/accounts/login/ajax/"
           with open('cookie.txt', 'rb') as s:
              #session = session.cookies.update(pickle.load(s))
              cookies = requests.utils.cookiejar_from_dict(pickle.load(s))
              #pickle.dump(a, s)
              #cookies = requests.utils.cookiejar_from_dict(pickle.load(s))
              session.cookies.update(cookies)
              #session = session(cookies=cookies) 
              #cookies = pickle.load(s)
              #if cookies:
              #jar = requests.cookies.RequestsCookieJar()
              #jar._cookies = cookies
              #session.cookies = jar
              print("session.cookies", session.cookies)
              #session.hooks['response'].append(cookie)
              #csrf_token = "2BYlKYXjE31YtbbT2KB8mRJ16ikszq14"
              #csrf_token = data[csrftoken]
              csrf_token = session.cookies['csrftoken']
              #data = {"username": login, "password": password}
        
              headers = {
                    "X-CSRFToken": csrf_token,
                    "referer": "https://www.instagram.com/",
                       }
              response = session.post(LOGIN_URL, headers=headers, cookies=cookies)
              #response = session.post("https://www.instagram.com/accounts/login/ajax/")
              response.raise_for_status()
              session.cookies.update(response.cookies)
              
        else:
            print("2")
            csrf_token = "23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv"
            login = "melvynasarterca"
            password = "00pp0099"
            data = {"username": login, "password": password}
        
            if "headers" in settings:
                #settings["params"] = {"query_hash": query_hash}
                settings["headers"]["X-CSRFToken"] = csrf_token
                settings["headers"]["referer"] = "https://www.instagram.com/"
            else:
                settings["headers"] = {
                    "X-CSRFToken": csrf_token,
                    "referer": "https://www.instagram.com/",
                       }

            response = session.post("https://www.instagram.com/accounts/login/ajax/", data, **settings)
            #response.raise_for_status()
            
            
            resp = response.json()
            field = 'checkpoint_url'
            
            if field in resp:
              print("11111")
              resp = response.json()[field]
              resp = "https://www.instagram.com" + resp
              data = {"choice": 1}
              response = session.post(resp, data, **settings)
              #print(response.text)
              response.raise_for_status()
              code = input('Enter code received: ')
              code_data = {'security_code': code}
              
              response = session.post(resp, code_data, **settings, allow_redirects=True)
              session.headers.update({'X-CSRFToken': response.cookies['csrftoken']})
              print("response")
            
            print("finish")

            # Save cookie
            with open("cookie.txt", 'wb') as fout:
              #f.truncate()
              pickle.dump(requests.utils.dict_from_cookiejar(session.cookies), fout)  #session.cookies._cookies
              print("f")
            print("response2:", response)
            try:
                data = response.json()
                if not data["authenticated"]:
                  raise AuthException(login) 
            except (ValueError, KeyError) as exception:
                 print("ERROR1")

AgentAccount(load=True)
#AgentAccount()

1
session.cookies <RequestsCookieJar[<Cookie csrftoken=se2F5mbejVPYiI8MtX0ZVHewm9XD3XcG for />, <Cookie ds_user_id=10387214183 for />, <Cookie mid=XG5wpgAEAAGYzCWUwh6LbXSpJBKT for />, <Cookie rur=FRC for />, <Cookie sessionid=10387214183%3AtenEsHzENLxy6s%3A2 for />, <Cookie urlgen="{35.233.191.48: 15169}:1gwkkR:DgDKxYW34xLYxx_DFs9RSMyuWI4" for />]>


# Получение кода из почты

In [20]:
#now = datetime.now()
import imaplib
#from datetime import datetime

import datetime
now = datetime.datetime.now()
mail = imaplib.IMAP4_SSL('imap.mail.ru')
mail.login('rogeca@bk.ru', '66yy6677')
mail.select("inbox")
import email
import re

result, data = mail.uid('search', None, "ALL") # Выполняет поиск и возвращает UID писем.
latest_email_uid = data[0].split()[-1]
result, data = mail.uid('fetch', latest_email_uid, '(RFC822)')
email_body = data[0][1]
mail = email.message_from_bytes(email_body)
ggg = mail['From']
hhh = mail['Date']
print(ggg)
print(hhh)
#data = mail.get_payload()[0]
#data = mail.get_payload(decode = False)
data = str(mail)
hhh = str(hhh)
print(hhh)
result = re.findall('[0-9]{6}', data)
#result2 = re.findall('[Date:]\s\w{3}.\s(\d{1-2}\s\w{3}\s\d{4}\s\d{2}:\d{2}:\d{2})', date)
result2 = re.findall('\w{3}.\s(\d{1-2}\s\w{3}\s\d{4}\s\d{2}:\d{2}:\d{2})', hhh)
print(result[9])
#result2 = str(result2)[0]
print("result2", result2)
d = datetime.datetime.strptime(result2, '%d %b %Y %H:%M:%S')

period = d - now
print("{} дней  {} секунд".format(period.days, period.seconds))
if period.seconds > 30:
  print("true")
if d < now:
  print("d < now")
elif d > now:
  print("d > now")

"Instagram" <security@mail.instagram.com>
Thu, 21 Feb 2019 00:36:50 -0800
Thu, 21 Feb 2019 00:36:50 -0800
214976
result2 []


TypeError: ignored

In [50]:
from datetime import datetime
now = datetime.now()
datetime.strftime(datetime.now(), "%Y.%m.%d %H:%M")
print(datetime.strftime(datetime.now(), "%Y.%m.%d %H:%M"))

2019.02.22 10:34


In [4]:
from datetime import datetime
now = datetime.now()
period = d - now
print("Всего: {} секунд".format(period.total_seconds()))
print("{} дней  {} секунд".format(period.days, period.seconds))


Всего: -122734.352263 секунд
-2 дней  50065 секунд


# Сохранение cookies на компьютер

In [0]:
import os
#session.close()
#response.raise_for_status()
#session
print( os.listdir('/content') )
from google.colab import files
files.download('cookie.txt') 

['.config', 'cookie.txt', 'sample_data']


# Загрузка Cookie на сервер

In [0]:
from google.colab import files
files.upload()

Saving cookie.txt to cookie.txt


{'cookie.txt': b'\x80\x03}q\x00(X\t\x00\x00\x00csrftokenq\x01X \x00\x00\x00se2F5mbejVPYiI8MtX0ZVHewm9XD3XcGq\x02X\n\x00\x00\x00ds_user_idq\x03X\x0b\x00\x00\x0010387214183q\x04X\x03\x00\x00\x00midq\x05X\x1c\x00\x00\x00XG5wpgAEAAGYzCWUwh6LbXSpJBKTq\x06X\x03\x00\x00\x00rurq\x07X\x03\x00\x00\x00FRCq\x08X\t\x00\x00\x00sessionidq\tX \x00\x00\x0010387214183%3AtenEsHzENLxy6s%3A2q\nX\x06\x00\x00\x00urlgenq\x0bX;\x00\x00\x00"{35.233.191.48: 15169}:1gwkkR:DgDKxYW34xLYxx_DFs9RSMyuWI4"q\x0cu.'}

# Проверка директории

In [0]:
import os
print( os.listdir('/content') )

['.config', 'sample_data']


# Получение подписчиков


In [0]:
#query_hash:56066f031e6239f35a904ac20c9f37d9
#variables:{"id":"10180157718","include_reel":true,"fetch_mutual":false,"first":12,"after":""}

import hashlib
from time import sleep
from requests.exceptions import HTTPError
def get_followers(account=None, pointer=None, count=100, settings={}, limit=50):
  
        #update(account, settings)
        csrf_token = "23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv"
        rhx_gis = "fc2e73d4fd7dddcd31d28bea5cb2df59"
        #query_hash = "QVFENklrNmY2Y2JvRGhEa2k5UV9XZTVkRHcwSDB6OXM3dWhhZkVsSXpNdlZacEJQVU9QVkcybDQtaC11c04tSXN3cWlmUzVIN3NxeG1BR2phV0VYSU1fcw=="
        query_hash="37479f2b8209594dde7facb0d904896a"
        idx = "2120762022"

        if pointer is None:
            #variables_string = '{{"id":"{id}","include_reel":true,"fetch_mutual":false,"first":{first}}}'
            variables_string = '{{"id":"{id}","first":{first}}}'
        else:
            #variables_string = '{{"id":"{id}","include_reel":true,"fetch_mutual":false,"first":{first},"after":"{after}"}}'
            variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
        followers = []

        while True:
            data = {"first": min(limit, count), "id": idx}
            if not pointer is None:
                data["after"] = pointer
            settings["params"] = {"query_hash": query_hash}
            variables=variables_string.format(**data)
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            #response = session.get("https://www.instagram.com/graphql/query/", **settings)
            #response.raise_for_status()
            
            
           
            for i in range(0, 3):
              try:
                response = session.get("https://www.instagram.com/graphql/query/", **settings)
                print(response.headers)
                response.raise_for_status()

              except (HTTPError, ValueError, KeyError) as exception:

                    print("ERROR", exception.response.status_code)
  
                    print("sleep") 
                    sleep(6) 
                    print("end sleep")
                    continue
              break
            #data = response.json()
            data = response.json()["data"]["user"]["edge_followed_by"]
            edges = data["edges"]
            page_info = data["page_info"]
            #print(page_info)
            followers_count = data["count"]
                
            for index in range(min(len(edges), count)):
                    node = edges[index]["node"]
                    a = node["username"]
                    id = node["id"]
                    profile_pic_url = node["profile_pic_url"]
                    is_verified = node["is_verified"]
                    full_name = node["full_name"]
                    followers.append(a)
                    if len(followers) % 100 == 0:
                      print(len(followers))
                
            if page_info["has_next_page"]:
                    pointer = page_info["end_cursor"]
            else:
                    pointer = None
                
            if len(edges) < count and page_info["has_next_page"]:
                    count = count-len(edges)
                    variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
            else:
                    return followers, pointer
            
                
get_followers("likekate_")

{'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Language, Cookie, Accept-Encoding', 'Content-Language': 'en', 'Content-Encoding': 'gzip', 'Date': 'Thu, 21 Feb 2019 09:37:31 GMT', 'Strict-Transport-Security': 'max-age=2592000', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Sat, 01 Jan 2000 00:00:00 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'content-security-policy': "report-uri https://www.instagram.com/security/csp_report/; default-src 'self' https://www.instagram.com; img-src https: data: blob:; font-src https: data:; media-src 'self' blob: https://www.instagram.com https://*.cdninstagram.com https://*.fbcdn.net; manifest-src 'self' https://www.instagram.com; script-src 'self' https://instagram.com https://www.instagram.com https://*.www.instagram.com https://*.cdninstagram.com wss://www.instagram.com https://*.facebook.com https://*.fbcdn.net https://*.facebook.net 'unsafe-inline' 'unsafe-eval' blob:; style-src 'se

(['semihtezmen',
  'kate_skoraya',
  'kabirov_rinat',
  'erosswimwear',
  's.afanasyev_official',
  'abdulkadyrov85',
  'an_vladi',
  '_armo_m',
  'miss_xtell',
  'help_all_animal',
  'ahmad_pv4',
  'mr_usmaniy',
  'karal2001',
  'rocks_lovers__7070',
  'uri_zakay_',
  'oxygen_cocktails',
  'mertuluturk01',
  'a.ftme',
  'muscle_zh_90',
  'liza_belogubova',
  'deepakgurjar0010',
  'johnny.svapo',
  'dinc9518',
  'alishastervovich',
  'many_principles',
  'karina.naumova',
  'kazake2000',
  'diana_2101k',
  'juliyawww',
  'svetlana_antonova_90',
  'sirena_by',
  'tsarevskaya__',
  'amirmt.dj',
  'berezka_design',
  'ed.thealien',
  'pandorasml',
  'ernismalka1991',
  'dinlaoda',
  'mir_serebraa',
  'ismailcinar27',
  'karina2001m',
  'ghostmic',
  'look_world20',
  'kristiy_mo',
  'ana_basnina',
  'mariammohammad49',
  'stogniy_yanina',
  'elsis_em',
  'roland.herzog',
  'pavel_rechkin_official',
  'falirakisexybeach2',
  'edelveis502019',
  '13blak13',
  'smrsmr410',
  'timur_tigr707',

In [0]:
import hashlib
import json
import re
import requests
user = {
	"email" : "depsemt2009@yandex.ru",
	"username" : "andivanov2019",
	"password" : "Asus2046",
}
import time
import random
def login(user):

    url = 'https://www.instagram.com/'
    url_login = 'https://www.instagram.com/accounts/login/ajax/'
    s = requests.Session()      
    s.cookies.update({
        'sessionid': '',
        'mid': '',
        'ig_pr': '1',
        'ig_vw': '1920',
        'csrftoken': '',
        's_network': '',
        'ds_user_id': ''
    })
    login_post = {
        'username': user['username'],
        'password': user['password']
    }
    s.headers.update({
        'Accept-Language': 'en-US,en;q=0.5',
	'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Content-Length': '0',
        'Host': 'www.instagram.com',
        'Origin': 'https://www.instagram.com',
        'Referer': 'https://www.instagram.com/',
	'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:56.0) Gecko/20100101 Firefox/56.0',
        'X-Instagram-AJAX': '1',
        'X-Requested-With': 'XMLHttpRequest'
    })
    try:
        r = s.get(url)
    except Exception as e:
        print(e)
    s.headers.update({'X-CSRFToken': r.cookies['csrftoken']})
    time.sleep(5 * random.random())

    login = s.post(
            url_login, data=login_post, allow_redirects=True)
    s.headers.update({'X-CSRFToken': login.cookies['csrftoken']})
    csrftoken = login.cookies['csrftoken']
    ds_user_id = login.cookies['ds_user_id']
    sessionid = login.cookies['sessionid']
    time.sleep(5 * random.random())
    loginjson = json.loads(login.content.decode('utf-8'))
    return csrftoken,ds_user_id,sessionid


csrftoken, ds_user_id, sessionid = login(user)

In [0]:
followers

NameError: ignored

In [0]:
get_followers("likekate_", pointer="QVFDSGVsXzh6MjVkWW5Oc1dNQ1RVcFlkOU5WTDFBcHVoMGozQXpGVjJvU1Q3M1RPc3lhMDRwaS1SYm9uVjdFY2dkZmU1eWtScUZ6cXc3YURMT2k3RnZkZw==")

{'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Language, Cookie', 'Content-Language': 'en', 'Date': 'Thu, 07 Feb 2019 13:48:28 GMT', 'Strict-Transport-Security': 'max-age=604800', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Sat, 01 Jan 2000 00:00:00 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'content-security-policy': "report-uri https://www.instagram.com/security/csp_report/; default-src 'self' https://www.instagram.com; img-src https: data: blob:; font-src https: data:; media-src 'self' blob: https://www.instagram.com https://*.cdninstagram.com https://*.fbcdn.net; manifest-src 'self' https://www.instagram.com; script-src 'self' https://instagram.com https://www.instagram.com https://*.www.instagram.com https://*.cdninstagram.com wss://www.instagram.com https://*.facebook.com https://*.fbcdn.net https://*.facebook.net 'unsafe-inline' 'unsafe-eval' blob:; style-src 'self' https://*.www.instagram.com https://www.in

([], None)